In [54]:
import numpy as np
import itertools

In [2]:
a = [[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]]

b = [[13, 14, 15], [16, 17, 18]]

a = np.array(a, dtype=np.float32)
b = np.array([2], dtype=np.float32)
c = b/a
# c.shape
c

array([[[2.        , 1.        , 0.6666667 ],
        [0.5       , 0.4       , 0.33333334]],

       [[0.2857143 , 0.25      , 0.22222222],
        [0.2       , 0.18181819, 0.16666667]]], dtype=float32)

In [3]:
a = [1,2,3,5,7]
b = [5,7]
b = a[:-2]+b
c = a.copy()
c == a

True

In [4]:
def prod(lst):
    n_prod = 1
    for i in lst:
        n_prod *= i
    return n_prod

In [5]:
def is_broadcastable(shape1, shape2):
    max_ndim = max(len(shape1), len(shape2))
    shape1 = [1] * (max_ndim - len(shape1)) + shape1
    shape2 = [1] * (max_ndim - len(shape2)) + shape2
    for dim1, dim2 in zip(shape1, shape2):
        if dim1 != dim2 and dim1 != 1 and dim2 != 1:
            return False
    return True

In [6]:
def broadcast_shape(ori_shape, b_shape):
    if(not is_broadcastable(ori_shape, b_shape)):
        raise ValueError("Shape should be broadcastable")
    max_ndim = max(len(ori_shape), len(b_shape))
    ori_shape = [1] * (max_ndim - len(ori_shape)) + ori_shape
    b_shape = [1] * (max_ndim - len(b_shape)) + b_shape
    res_shape = []
    for dim1, dim2 in zip(ori_shape, b_shape):
        res_shape.append(max(dim1, dim2))
    return res_shape

In [94]:
def broadcast_array(arr, old_shape, target_shape):
    if len(target_shape) < len(old_shape):
        raise ValueError("ndim of target shape is less than original shape ndim")

    if len(old_shape) < len(target_shape):
        extra_dim_n = len(target_shape) - len(old_shape)
        for s1, s2 in zip(old_shape, target_shape[extra_dim_n:]):
            if (s1 != s2) and s1 != 1:
                raise ValueError("Invalid target shape")
    else:
        for s1, s2 in zip(old_shape, target_shape):
            if (s1 != s2) and s1 != 1:
                raise ValueError("Invalid target shape")
    # To store broadcasted array temprarily
    temp_result = []

    # Case: when array dim is less than target shape dim
    if len(old_shape) < len(target_shape):
        extra_dim_n = len(target_shape) - len(old_shape)
        old_shape = target_shape[:extra_dim_n] + old_shape
        repeat = prod(target_shape[:extra_dim_n])
        for _ in range(repeat):
            temp_result += arr.copy()

    # General case
    repeat_axes = []
    if len(temp_result) == 0:
        result = arr
    else:
        result = temp_result

    for i in range(len(old_shape)):
        if target_shape[i] > old_shape[i] and old_shape[i] == 1:
            repeat_axes.append(i)
            
    if len(repeat_axes) == 0:
        return result
 
    
    # print(strides)
    for axis in repeat_axes:
        # Calculate strides
        strides = [prod(old_shape[i+1:]) for i in range(len(old_shape))]
        sub_shape = old_shape[:axis+1]
        sub_strides = strides[:axis+1]
        print(f"old_shape:{old_shape}, strides:{strides}")
        print(f"sub_shape:{sub_shape}, sub_strides:{sub_strides}")
        indices = itertools.product(*[[i for i in range(n)] for n in sub_shape])
        temp_result = []
        for index in indices:
            print("index",index)
            start_idx = 0
            for n in [index[i]*sub_strides[i] for i in range(len(index))]:
                start_idx += n
            end_idx = start_idx + sub_strides[-1]
            print(f"start_idx:{start_idx}, end_idx:{end_idx}")
            print("resul sliced",result[start_idx:end_idx])
            for _ in range(target_shape[axis]):
                temp_result += result[start_idx:end_idx].copy()
            print("temp:",temp_result)
        old_shape[axis] = target_shape[axis]
        result = temp_result
        print("result",result)
    # print(indices, repeat_axes)
    return result

In [100]:
a = [1,2]
shape_a = [2,1]
target_shape_a = [2,2,2,4]

b = [1,2,3,4,5,6,7,8]
shape_b = [2,1,1,4]
target_shape_b = [2,2,2,4]
np.array(broadcast_array(b, shape_b, target_shape_b)).reshape(target_shape_b)
# np.array(broadcast_array(b, shape_b, target_shape_b)).reshape(2,2,2,1)
# broadcast_shape(shape, target_shape)

old_shape:[2, 1, 1, 4], strides:[4, 4, 4, 1]
sub_shape:[2, 1], sub_strides:[4, 4]
index (0, 0)
start_idx:0, end_idx:4
resul sliced [1, 2, 3, 4]
temp: [1, 2, 3, 4, 1, 2, 3, 4]
index (1, 0)
start_idx:4, end_idx:8
resul sliced [5, 6, 7, 8]
temp: [1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8]
result [1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8]
old_shape:[2, 2, 1, 4], strides:[8, 4, 4, 1]
sub_shape:[2, 2, 1], sub_strides:[8, 4, 4]
index (0, 0, 0)
start_idx:0, end_idx:4
resul sliced [1, 2, 3, 4]
temp: [1, 2, 3, 4, 1, 2, 3, 4]
index (0, 1, 0)
start_idx:4, end_idx:8
resul sliced [1, 2, 3, 4]
temp: [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
index (1, 0, 0)
start_idx:8, end_idx:12
resul sliced [5, 6, 7, 8]
temp: [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8]
index (1, 1, 0)
start_idx:12, end_idx:16
resul sliced [5, 6, 7, 8]
temp: [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8]
result [1, 2, 3, 4, 1, 2, 3, 4, 1

array([[[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4]]],


       [[[5, 6, 7, 8],
         [5, 6, 7, 8]],

        [[5, 6, 7, 8],
         [5, 6, 7, 8]]]])

In [109]:
a = np.array(a).reshape(2,1)
b = np.array(b).reshape(2,1,1,4)
np.broadcast_to(b, target_shape_b)
# a + b 

array([[[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4]]],


       [[[5, 6, 7, 8],
         [5, 6, 7, 8]],

        [[5, 6, 7, 8],
         [5, 6, 7, 8]]]])

In [68]:
def test(a):
    return a +5

In [70]:
b = 5
b, test(b)


(5, 10)